# FRR Fabric CERN

Start with validation of config and ssh key

FRR, SiteRM, XrootD on Fabric

This setup will install the following:
* 1 Router VM with FRR (either Software one or with VPP).
* 4 Hosts (XRootD) and all will point as GW to Router;
* SiteRM Frontend (optional) + Agents will be installed on the Router and Host VMs
* Monitoring node with Grafana/Prometheus and full ELStack.

In [13]:
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
from random import randint


from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import mflib 
from mflib.mflib import MFLib


# Replace me with your project ID;
project_id = "a57c7715-d871-4369-82e6-408c9a57a6e7"
fablib = fablib_manager(project_id=project_id)

fablib = fablib_manager()
fablib.show_config();
# Verify and configure client.
fablib.verify_and_configure()
# Save configuration
fablib.save_config()
# Check version
print(f"MFLib version  {mflib.__version__} " )


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,a57c7715-d871-4369-82e6-408c9a57a6e7
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jbalcas_0000188368
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key1
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


User: jbalcas@es.net bastion key is valid!
Configuration is valid and please save the config!
MFLib version  1.0.6 


## Create the Experiment Slice

# Configuration parameters

In [14]:
site = 'CIEN'
slice_name = f'FRR-{site.lower()}'
routername = f'frr-{site.lower()}'
image_name = "docker_ubuntu_22"
host_image = 'docker_ubuntu_22'


site_ranges = {'CERN': '2602:fcfb:001d',
               'LOSA': '2602:fcfb:0012',
               'NEWY': '2602:fcfb:0013',
               'UCSD': '2602:fcfb:000d',
               'CIEN': '2602:fcfb:0023'}
router_asns = {'CERN': 64518,
               'LOSA': 64519,
               'NEWY': 64520,
               'UCSD': 64521,
               'CIEN': 64523}

routerasn = router_asns[site]
rangestart = site_ranges[site]


# fff0::/64 - is used for Peering with Cisco Fabric;
frr_config = {'range': f'{rangestart}:fff0::/64',
              'ip':  f'{rangestart}:fff0::2/64',
              'gw':  f'{rangestart}:fff0::1',
              'vlan': 99}

# :fff1::64, :fff2::/64, :fff3::/64 
# Those IP Ranges uses for FRR node
ip_ranges = { f'{rangestart}:fff1::/64': {
    'frr':  f'{rangestart}:fff1::1/64',
    'vlan': 100,
    'name': 'frr-net0'},
         f'{rangestart}:fff2::/64': {
    'frr':  f'{rangestart}:fff2::1/64',
    'vlan': 101,
    'name': 'frr-net1'},
         f'{rangestart}:fff3::/64': {
    'frr':  f'{rangestart}:fff3::1/64',
    'vlan': 102,
    'name': 'frr-net2'}}

# Used to map same frr network to iprange used on host
ip_range_to_net = {'host1_n1': 'frr-net0',
                  'host1_n2': 'frr-net1',
                  'host1_n3': 'frr-net2',
                  'host2_n1': 'frr-net0',
                  'host2_n2': 'frr-net1',
                  'host2_n3': 'frr-net2',
                  'host3_n1': 'frr-net0',
                  'host3_n2': 'frr-net1',
                  'host3_n3': 'frr-net2',
                  'host4_n1': 'frr-net0',
                  'host4_n2': 'frr-net1',
                  'host4_n3': 'frr-net2'
                  }

vlan_to_net = {'host1_n1': 100,
                  'host1_n2': 101,
                  'host1_n3': 102,
                  'host2_n1': 100,
                  'host2_n2': 101,
                  'host2_n3': 102,
                  'host3_n1': 100,
                  'host3_n2': 101,
                  'host3_n3': 102,
                  'host4_n1': 100,
                  'host4_n2': 101,
                  'host4_n3': 102
              }


# Host IPs and corresponding gateways (pointing to FRR)
hosts = {"host1": {
    'host1_n1': { f'{rangestart}:fff1::2/64':  f'{rangestart}:fff1::1'},
    'host1_n2': { f'{rangestart}:fff2::2/64':  f'{rangestart}:fff2::1'},
    'host1_n3': { f'{rangestart}:fff3::2/64':  f'{rangestart}:fff3::1'},
}, "host2": {
    'host2_n1': { f'{rangestart}:fff1::3/64':  f'{rangestart}:fff1::1'},
    'host2_n2': { f'{rangestart}:fff2::3/64':  f'{rangestart}:fff2::1'},
    'host2_n3': { f'{rangestart}:fff3::3/64':  f'{rangestart}:fff3::1'},
}, "host3": {
    'host3_n1': { f'{rangestart}:fff1::4/64':  f'{rangestart}:fff1::1'},
    'host3_n2': { f'{rangestart}:fff2::4/64':  f'{rangestart}:fff2::1'},
    'host3_n3': { f'{rangestart}:fff3::4/64':  f'{rangestart}:fff3::1'},
}, "host4": {
    'host4_n1': { f'{rangestart}:fff1::5/64':  f'{rangestart}:fff1::1'},
    'host4_n2': { f'{rangestart}:fff2::5/64':  f'{rangestart}:fff2::1'},
    'host4_n3': { f'{rangestart}:fff3::5/64':  f'{rangestart}:fff3::1'},
}}


# List of destinations, for which to add static routes and use dataplane interface
# Most are needed for XRootD communications, except:
#2620:f:a:50::/64 - Nycernet - for communications with FE running on NRP (Issue with SENSE-O and IPv6)
#2601:0249:187f:cf74::/64 - Home Range to access Frontpage
use_dataplane_routes = {"2602:fcfb::/36": "Fabric",
                        "2001:1458::/32": "Cern",
                        "2620:6a::/48": "Fermilab",
                        "2605:d9c0::/32": "Caltech",
                        "2600:900::/28": "Nebraska",
                        "2001:48d0::/32": "SDSC",
                        "2620:f:a:50::/64": "NyCerNet",
                        "2600:2701:5000:5001::/64": "NyCernet1",
                        "2601:0249:187f:cf74::/64": "JustasHome",
                        "2607:f720::/32": "UCSD"}

# XRootd configurations
# Location of XRootD Redirector (only passed as env parameter to xrootd)
xrd_redir = f"{site.lower()}-sub1-host1-redir.exp.fabric-testbed.net"
# XRootD HTTP Requires to provide secret (and must be equal between all xrootd servers).
# Modify as you see it fit.
xrd_http_secret = "pYrySWhj4BftwJkSbMyAk8ha3p5YXsAt7g3mFzx7Vkg"


# Helper Functions
import subprocess
import shlex

def runcmd(nodes, commands):
    print("RUN COMMANDS")
    for command in commands:
        for nodename in nodes:
            node = slice.get_node(name=nodename)
            print(f'Execute: {command} on {nodename}')
            stdout, stderr = node.execute(command)

def runonecmd(node, nodename, command):
    print(f'Execute: {command} on {nodename}')
    stdout, stderr = node.execute(command)

def runIntfTuning(nodes):
    print("NODE INTERFACE TUNING")
    for nodename in nodes:
        node = slice.get_node(name=nodename)
        for intf in node.get_interfaces():
            command = f"sudo sh vpp-frr/interface-tuning.sh {intf.get_device_name()}"
            print(f'Execute: {command} on {nodename}')
            stdout, stderr = node.execute(command)

def localcmd(command):
    """Execute command locally and return stdout and stderr."""
    print(f'Execute local cmd: {command}')
    command = shlex.split(str(command))
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return proc.communicate()

# Create Slice

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Add Router;
router = slice.add_node(name=routername, site=site, cores=16, ram=32, disk=100, image=image_name)
if site in ['CERN', 'CIEN']:
    routernic1 = router.add_component(model='NIC_ConnectX_6', name='nic_local1').get_interfaces()
    slice.add_l2network(name="nic_local1_1", interfaces=[routernic1[0]])
    slice.add_l2network(name="nic_local1_2", interfaces=[routernic1[1]])

routernic = router.add_component(model='NIC_ConnectX_6', name='nic_local').get_interfaces()
# This will get dedicated NIC with 2 ports.
# Port-1 - Is used for Peering with Fabric Cisco
# Port-2 - Is used for hosts (L2 Network)
# For SENSE Needs - everything uses sub-interfaces;
subnet = IPv6Network(frr_config['range'])
vlan = frr_config['vlan']
net1 = slice.add_l3network(name=f'pub-0', type='IPv6Ext', subnet=subnet)
ciface = routernic[1].add_sub_interface(f"vlan{vlan}", vlan=str(vlan), bw=100)
ciface.set_mode('manual')
net1.add_interface(ciface)

# Add Networks and interfaces to FRR
for iprange, rangeitems in ip_ranges.items():
    subnet = IPv6Network(iprange)
    vlan = rangeitems['vlan']
    gateway = subnet[1]
    net1 = slice.add_l2network(name=rangeitems['name'], subnet=subnet, gateway=gateway)
    ciface = routernic[0].add_sub_interface(f"vlan{vlan}", vlan=str(vlan), bw=100)
    ciface.set_mode('manual')
    net1.add_interface(ciface)

# Add Network and interfaces to Hosts
for host, rangeitems in hosts.items():
    node = slice.add_node(name=f"{host}", site=site, cores=16, ram=32, disk=1000, image=host_image)
    counter = 0
    for hostnet, hostrange in rangeitems.items():
        for hostip, hostgw in hostrange.items():
            iface = node.add_component(model='NIC_Basic', name=hostnet).get_interfaces()[0]
            iface.set_mode('manual')
            net1 = slice.get_network(name=ip_range_to_net[hostnet])
            net1.add_interface(iface)

# Add Measurement node:
MFLib.addMeasNode(slice, site=site, disk=100)

slice.submit()


ID,9879cc7b-e0e3-4643-8874-359c8607397d
Name,FRR-cien
Lease Expiration (UTC),2025-04-01 22:18:30 +0000
Lease Start (UTC),2025-03-31 22:18:30 +0000
Project ID,a57c7715-d871-4369-82e6-408c9a57a6e7
State,StableError
Email,jbalcas@es.net
UserId,91f5ecc3-16ff-4f09-95ac-dfeee0c3b1e3


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
98bf8458-da61-460d-9e80-2161a94ddf76,frr-cien,16,32,100,docker_ubuntu_22,qcow2,cien-w2.fabric-testbed.net,CIEN,ubuntu,163.253.48.242,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@163.253.48.242,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
861f6ac0-c31a-4200-9285-41e5bb3aaf85,host1,16,32,1000,docker_ubuntu_22,qcow2,cien-w1.fabric-testbed.net,CIEN,ubuntu,163.253.48.230,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@163.253.48.230,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
4a650287-c7d3-4c77-a4cb-8709953a201d,host2,16,32,1000,docker_ubuntu_22,qcow2,cien-w1.fabric-testbed.net,CIEN,ubuntu,163.253.48.247,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@163.253.48.247,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
63ea3dde-ec2b-41d5-892b-cf9b3b2e5367,host3,16,32,1000,docker_ubuntu_22,qcow2,cien-w1.fabric-testbed.net,CIEN,ubuntu,163.253.48.236,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@163.253.48.236,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d7f8feca-6972-4ada-ac88-ac8d30653bbd,host4,16,32,1000,docker_ubuntu_22,qcow2,cien-w1.fabric-testbed.net,CIEN,ubuntu,163.253.48.253,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@163.253.48.253,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
840e39de-e5be-4aae-afc8-752ae28c54d1,meas-node,4,16,100,docker_ubuntu_20,qcow2,cien-w1.fabric-testbed.net,CIEN,ubuntu,163.253.48.240,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@163.253.48.240,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
96b61c07-7d3c-40bc-9b8a-928a87b1d00e,frr-net0,L2,L2Bridge,CIEN,frr-net0.subnet,frr-net0.gateway,Failed,"failed lease update- all units failed priming: Exception during create for unit: 96b61c07-7d3c-40bc-9b8a-928a87b1d00e Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device cien-data-sw: Prepare error: RPC error: error bad-attribute: unknown namespace (error-path: /nc:rpc/nc:edit-config/nc:config/classifier:classifiers/classifier:classifier[classifier:name=vlan-2031]), internal: jsonrpc_tx_commit395#all units failed priming: Exception during create for unit: 96b61c07-7d3c-40bc-9b8a-928a87b1d00e Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device cien-data-sw: Prepare error: RPC error: error bad-attribute: unknown namespace (error-path: /nc:rpc/nc:edit-config/nc:config/classifier:classifiers/classifier:classifier[classifier:name=vlan-2031]), internal: jsonrpc_tx_commit395#"
45db4cc5-19e1-4c68-9efc-4ad7d6b694ce,frr-net1,L2,L2Bridge,CIEN,frr-net1.subnet,frr-net1.gateway,Failed,"failed lease update- all units failed priming: Exception during create for unit: 45db4cc5-19e1-4c68-9efc-4ad7d6b694ce Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device cien-data-sw: Prepare error: RPC error: error bad-attribute: unknown namespace (error-path: /nc:rpc/nc:edit-config/nc:config/classifier:classifiers/classifier:classifier[classifier:name=vlan-2014]), internal: jsonrpc_tx_commit395#all units failed priming: Exception during create for unit: 45db4cc5-19e1-4c68-9efc-4ad7d6b694ce Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device cien-data-sw: Prepare error: RPC error: error bad-attribute: unknown namespace (error-path: /nc:rpc/nc:edit-config/nc:config/classifier:classifiers/classifier:classifier[classifier:name=vlan-2014]), internal: jsonrpc_tx_commit395#"
15a75f64-668b-4c87-82ea-c756a483807f,frr-net2,L2,L2Bridge,CIEN,frr-net2.subnet,frr-net2.gateway,Closed,"failed lease update- all units failed priming: Exception during create for unit: 15a75f64-668b-4c87-82ea-c756a483807f Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device cien-data-sw: Prepare error: RPC error: error bad-attribute: unknown namespace (error-path: /nc:rpc/nc:edit-config/nc:config/classifier:classifiers/classifier:classifier[classifier:name=vlan-2055]), internal: jsonrpc_tx_commit395#all units failed priming: Exception during create for unit: 15a75f64-668b-4c87-82ea-c756a483807f Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device cien-data-sw: Prepare error: RPC error: error bad-attribute: unknown namespace (error-path: /nc:rpc/nc:edit-config/nc:config/classifier:classifiers/classifier:classifier[classifier:name=vlan-2055]), internal: jsonrpc_tx_commit395#"
92e7ee42-ebed-430e-8cbc-5663e09227e0,l3_meas_net_CIEN,L3,FABNetv4,CIEN,l3_meas_net_CIEN.subnet,l3_meas_net_CIEN.gateway,Active,
0ff3c3d1-144d-4362-9f12-cdc525a49bfc,nic_local1_1,L2,L2Bridge,CIEN,nic_local1_1.subnet,nic_local1_1.gateway,Active,
d8e198ff-3c9f-4f0b-b3f4-c64435712f4b,nic_local1_2,L2,L2Bridge,CIEN,nic_local1_2.subnet,nic_local1_2.gateway,Active,
88570099-994b-4daa-ae6a-309f993b206e,pub-0,L3,FABNetv6Ext,CIEN,pub-0.subnet,pub-0.gatewa


Time to StableError 271 seconds
Running post_boot_config ... 


## Deploy monitoring (Takes long time to install)

In [ ]:
print(f"MFLib version  {mflib.__version__} " )
# Import MFLib Class
from mflib.mflib import MFLib
# The slice name of the slice with which you want to interact.
mf = MFLib(slice_name, mf_repo_branch="main")
print(mf.grafana_tunnel)
instrumetize_results = mf.instrumentize()
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

# All node tuning for high throughput

In [ ]:
slice = fablib.get_slice(name=slice_name)

commands = ["sudo apt update",
            "sudo apt install -y docker docker-compose-plugin", 
            "sudo usermod -aG docker ubuntu",
            "git clone https://github.com/sdn-sense/vpp-frr",
            "sudo sh vpp-frr/tuning.sh"]
runcmd([routername], commands)
print('-'*100)
runIntfTuning([routername])
print('Run tuning on all hosts')
runcmd(list(hosts.keys()), commands)
print('-'*100)
runIntfTuning(list(hosts.keys()))

## Configure IPs on Hosts (set GWs)

In [ ]:
# Set IPs on Hosts;
for host, rangeitems in hosts.items():
    node1 = slice.get_node(name=host)
    counter = 0
    intfdone = []
    for hostnet, hostrange in rangeitems.items():
        for hostip, hostgw in hostrange.items():
            node1_iface = node1.get_interface(name=f'{host}-{hostnet}-p1')
            intf_name = node1_iface.get_device_name()
            net1 = slice.get_network(name=ip_range_to_net[hostnet])
            if intf_name not in intfdone:
                runonecmd(node1, host, f"sudo sysctl -w net.ipv6.conf.{intf_name.replace('.', '/')}.accept_ra=0")
                runonecmd(node1, host, f"sudo ip link set {intf_name} down")
                runonecmd(node1, host, f"sudo ip link set {intf_name} up")
                intfdone.append(intf_name)
            command = f"sudo ip addr add {hostip} dev {intf_name}"
            runonecmd(node1, host, command)

In [ ]:
# Create routing tables and set default gws for each.
for host, rangeitems in hosts.items():
    node1 = slice.get_node(name=host)
    counter = 1
    intfdone = []
    for hostnet, hostrange in rangeitems.items():
        for hostip, hostgw in hostrange.items():
            node1_iface = node1.get_interface(name=f'{host}-{hostnet}-p1')
            intf_name = node1_iface.get_device_name()
            hostipnomask = hostip.split('/')[0]
            hostgwnomask = hostgw.split('/')[0]
            runonecmd(node1, host, f'grep -qF "{intf_name}" /etc/iproute2/rt_tables || echo "{counter} {intf_name}" | sudo tee -a /etc/iproute2/rt_tables > /dev/null')
            runonecmd(node1, host, f'sudo ip -6 r add {hostipnomask} dev {intf_name} table {intf_name}')
            runonecmd(node1, host, f'sudo ip -6 route add default via {hostgwnomask} dev {intf_name} table {intf_name}')
            runonecmd(node1, host, f'sudo ip -6 rule add from {hostipnomask}/128 table {intf_name}')
            runonecmd(node1, host, f'sudo ip -6 rule add to {hostipnomask}/128 table {intf_name}')
            runonecmd(node1, host, f'sudo ping6 -c2 {hostgwnomask}')
            counter += 1

# (Option 1) Deploy FRR and configure FRR to be a router. (No-DPDK)

## Set IPs on Router

In [ ]:
slice = fablib.get_slice(name=slice_name)
router = slice.get_node(name=routername)
# 0. Stop any docker containers if running
runonecmd(router, routername, f"docker stop vpp")
runonecmd(router, routername, f"docker stop frr")
# 1. Disable accept_ra for the first interface. It adds a default route and we want to avoid that
router_iface = router.get_interface(network_name=f'pub-0')
iface = router_iface.get_physical_os_interface_name()
runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
runonecmd(router, routername, f"sudo ip link set {iface} down")
runonecmd(router, routername, f"sudo ip link set {iface} up")
# 2. Disable accept_ra also for vlan device
iface = router_iface.get_device_name()
runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
runonecmd(router, routername, f"sudo ip link set {iface} down")
runonecmd(router, routername, f"sudo ip link set {iface} up")
# 3. Set IP and default routes as defined in configuration
#   Issue on Fabric IPv6 Mgmt hosts - default route will be via mgmt;
# Deleting it will restore back. Would need explicit routing, but at the same time
# need a way to control mgmt interface setup. Best would be if there is an option
# to say that mgmt interface - do not set default route and use only explicit routing.
# If we want to make it static - we also need to modify netplan (which is also controlled by fabric)
iface = router_iface.get_device_name()
runonecmd(router, routername, f"sudo ip link add link {iface.split('.')[0]} name {iface} type vlan id {iface.split('.')[1]}")
runonecmd(router, routername, f"sudo ip link set {iface} up")
command = f"sudo ip addr add {frr_config['ip']} dev {iface}"
runonecmd(router, routername, command)


# For all other L2 networks for Hosts - Set interfaces up and add correct IPs
counter=0
intfdone = []
for iprange, rangeitems in ip_ranges.items():
    router_iface = router.get_interface(network_name=rangeitems['name'])
    # Disable accept_ra
    iface = router_iface.get_physical_os_interface_name()
    if iface not in intfdone:
        runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
        runonecmd(router, routername, f"sudo ip link set {iface} down")
        runonecmd(router, routername, f"sudo ip link set {iface} up")
        intfdone.append(iface)
    # Disable accept_ra for sub device
    iface = router_iface.get_device_name()
    if iface not in intfdone:
        runonecmd(router, routername, f"sudo ip link add link {iface.split('.')[0]} name {iface} type vlan id {iface.split('.')[1]}")
        runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
        runonecmd(router, routername, f"sudo ip link set {iface} down")
        runonecmd(router, routername, f"sudo ip link set {iface} up")
        intfdone.append(iface)
    command = f"sudo ip addr add {rangeitems['frr']} dev {iface}"
    runonecmd(router, routername, command)
    counter+=1

## Deploy FRR

In [ ]:
slice = fablib.get_slice(name=slice_name)
router = slice.get_node(name=routername)

# If we have frrsshkey and frrsshkey.pub on Jupyter notebook dir, we copy it to router machine.
# Otherwise generate new one. In case new one, SiteRM FE will need to update credentials to access device.
if os.path.isfile('frrsshkey') and os.path.isfile('frrsshkey.pub'):
    runcmd([routername], ["mkdir -p ~/.ssh", "chmod 700 ~/.ssh"])
    scp_cmd = "scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config "
    user = router.get_username()
    mgmtip = router.get_management_ip()
    for dest, srcfile in {'~/.ssh/frrsshkey': 'frrsshkey',
                          '~/.ssh/frrsshkey.pub': 'frrsshkey.pub'}.items():
        cmd = f"{scp_cmd} {srcfile} {user}@[{mgmtip}]:{dest}"
        print(localcmd(cmd))
    runcmd([routername], ["chmod 600 ~/.ssh/frrsshkey"])

# start frr container
commands = ["sudo nft flush ruleset",
    f'cd vpp-frr/frr/ && sed -e "s/REPLACEME_HOSTNAME/`hostname`/" -e "s/REPLACEME_ASN/{routerasn}/" etc/frr/frr.conf-template > etc/frr/frr.conf',
    "cd vpp-frr/frr/ && sudo docker compose up -d",
    "sh vpp-frr/frr/keygen.sh"]
runcmd([routername], commands)

## Add default routes

In [ ]:
slice = fablib.get_slice(name=slice_name)
# This is default routes, once VPP, FRR is running;
router = slice.get_node(name=routername)
# 1. Disable accept_ra for the first interface. That add a default route and we want to avoid that
router_iface = router.get_interface(network_name=f'pub-0')
iface = router_iface.get_physical_os_interface_name()
# 4. Set default routes for all our known ranges (CERN,Fabric,Caltech,SDSC,Fermilab,Nebraska, etc..)
for iprange, name in use_dataplane_routes.items():
    command = f"sudo ip -6 route add {iprange} via {frr_config['gw']} dev {iface}.{frr_config['vlan']}"
    print(command)
    runonecmd(router, routername, command)

# (Option 2) Deploy FRR with VPP and configure FRR to be a router. (DPDK)

## Disable accept_ra on all interfaces, set them down, remove vlans

In [21]:
slice = fablib.get_slice(name=slice_name)
router = slice.get_node(name=routername)
# 0. Stop any docker containers if running
runonecmd(router, routername, f"docker stop vpp")
runonecmd(router, routername, f"docker stop frr")

# 1. Disable accept_ra for the first interface. That add a default route and we want to avoid that
router_iface = router.get_interface(network_name=f'pub-0')
iface = router_iface.get_physical_os_interface_name()
runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
runonecmd(router, routername, f"sudo ip link set {iface} down")
# 2. Disable accept_ra also for vlan device
iface = router_iface.get_device_name()
runonecmd(router, routername, f"sudo ip link del {iface}")

# For all other L2 networks for Hosts - Delete interfaces as needed by VPP.
counter=0
intfdone = []
for iprange, rangeitems in ip_ranges.items():
    router_iface = router.get_interface(network_name=rangeitems['name'])
    # Disable accept_ra
    iface = router_iface.get_physical_os_interface_name()
    if iface not in intfdone:
        runonecmd(router, routername, f"sudo sysctl -w net.ipv6.conf.{iface.replace('.', '/')}.accept_ra=0")
        runonecmd(router, routername, f"sudo ip link set {iface} down")
        intfdone.append(iface)
    # Disable accept_ra for sub device
    iface = router_iface.get_device_name()
    if iface not in intfdone:
        runonecmd(router, routername, f"sudo ip link del {iface}")
        intfdone.append(iface)
    counter+=1

Execute: docker stop vpp on frr-cern
vpp
Execute: docker stop frr on frr-cern
frr
Execute: sudo sysctl -w net.ipv6.conf.enp11s0np0.accept_ra=0 on frr-cern
net.ipv6.conf.enp11s0np0.accept_ra = 0
Execute: sudo ip link set enp11s0np0 down on frr-cern
Execute: sudo ip link del enp11s0np0.99 on frr-cern
Cannot find device "enp11s0np0.99"
Execute: sudo sysctl -w net.ipv6.conf.enp10s0np0.accept_ra=0 on frr-cern
net.ipv6.conf.enp10s0np0.accept_ra = 0
Execute: sudo ip link set enp10s0np0 down on frr-cern
Execute: sudo ip link del enp10s0np0.100 on frr-cern
Cannot find device "enp10s0np0.100"
Execute: sudo ip link del enp10s0np0.101 on frr-cern
Cannot find device "enp10s0np0.101"
Execute: sudo ip link del enp10s0np0.102 on frr-cern
Cannot find device "enp10s0np0.102"


## Deploy VPP

In [22]:
# Default commands
commands = ["sudo nft flush ruleset",
            f"cd vpp-frr && git pull",
            f"cp vpp-frr/siterm/{site}/fe/startup.gate vpp-frr/vpp/etc/vpp/startup.gate",
            f"cp vpp-frr/siterm/{site}/fe/run-routeadd.sh vpp-frr/vpp/root/run-routeadd.sh"]

if site in ['CERN', 'CIEN']:
    commands.append(f"cp vpp-frr/siterm/{site}/fe/startup.conf-template vpp-frr/vpp/etc/vpp/startup.conf-template")

# Identify number of cores and modify the env file to reflect based on cores;
commands.append(f"cd vpp-frr/vpp/ && sh update_cpus.sh")

# Identify public interface and add it to env;
slice = fablib.get_slice(name=slice_name)
router = slice.get_node(name=routername)
# 1. Disable accept_ra for the first interface. That add a default route and we want to avoid that
router_iface = router.get_interface(network_name=f'pub-0')
pub_iface = router_iface.get_physical_os_interface_name()
commands.append(f"cd vpp-frr/vpp/ && sh find_interfaces.sh {pub_iface} PUBLIC")

# Identify private interface and add it to env file;
# For all other L2 networks for Hosts - Move interfaces on FRR also to Net-NS and set IPs;
counter=0
for iprange, rangeitems in ip_ranges.items():
    router_iface = router.get_interface(network_name=rangeitems['name'])
    # Disable accept_ra
    priv_iface = router_iface.get_physical_os_interface_name()
    break
commands.append(f"cd vpp-frr/vpp/ && sh find_interfaces.sh {priv_iface} PRIVATE")

if site in ['CERN', 'CIEN']:
    # Public 1 as e3
    router_iface = router.get_interface(network_name=f'nic_local1_1')
    pub_iface1 = router_iface.get_physical_os_interface_name()
    commands.append(f"cd vpp-frr/vpp/ && sh find_interfaces.sh {pub_iface1} PUBLIC1")
    # Public 2 as e4
    router_iface = router.get_interface(network_name=f'nic_local1_2')
    pub_iface2 = router_iface.get_physical_os_interface_name()
    commands.append(f"cd vpp-frr/vpp/ && sh find_interfaces.sh {pub_iface2} PUBLIC2")

commands.append("cd vpp-frr/vpp/ && sh start.sh")
print(commands)

runcmd([routername], commands)

['sudo nft flush ruleset', 'cd vpp-frr && git pull', 'cp vpp-frr/siterm/CERN/fe/startup.gate vpp-frr/vpp/etc/vpp/startup.gate', 'cp vpp-frr/siterm/CERN/fe/run-routeadd.sh vpp-frr/vpp/root/run-routeadd.sh', 'cp vpp-frr/siterm/CERN/fe/startup.conf-template vpp-frr/vpp/etc/vpp/startup.conf-template', 'cd vpp-frr/vpp/ && sh update_cpus.sh', 'cd vpp-frr/vpp/ && sh find_interfaces.sh enp11s0np0 PUBLIC', 'cd vpp-frr/vpp/ && sh find_interfaces.sh enp10s0np0 PRIVATE', 'cd vpp-frr/vpp/ && sh find_interfaces.sh enp8s0np0 PUBLIC1', 'cd vpp-frr/vpp/ && sh find_interfaces.sh enp9s0np0 PUBLIC2', 'cd vpp-frr/vpp/ && sh start.sh']
RUN COMMANDS
Execute: sudo nft flush ruleset on frr-cern
Execute: cd vpp-frr && git pull on frr-cern
Already up to date.
Execute: cp vpp-frr/siterm/CERN/fe/startup.gate vpp-frr/vpp/etc/vpp/startup.gate on frr-cern
Execute: cp vpp-frr/siterm/CERN/fe/run-routeadd.sh vpp-frr/vpp/root/run-routeadd.sh on frr-cern
Execute: cp vpp-frr/siterm/CERN/fe/startup.conf-template vpp-frr/vpp

## Deploy FRR

In [23]:
slice = fablib.get_slice(name=slice_name)
router = slice.get_node(name=routername)

# If we have frrsshkey and frrsshkey.pub on Jupyter notebook dir, we copy it to router machine.
# Otherwise generate new one. In case new one, SiteRM FE will need to update credentials to access device.
if os.path.isfile('frrsshkey') and os.path.isfile('frrsshkey.pub'):
    runcmd([routername], ["mkdir -p ~/.ssh", "chmod 700 ~/.ssh"])
    scp_cmd = "scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config "
    user = router.get_username()
    mgmtip = router.get_management_ip()
    for dest, srcfile in {'~/.ssh/frrsshkey': 'frrsshkey',
                          '~/.ssh/frrsshkey.pub': 'frrsshkey.pub'}.items():
        cmd = f"{scp_cmd} {srcfile} {user}@[{mgmtip}]:{dest}"
        print(localcmd(cmd))
    runcmd([routername], ["chmod 600 ~/.ssh/frrsshkey"])


# Flush nftables and start frr container
commands = ["sudo nft flush ruleset",
    f'cd vpp-frr/frr/ && sed -e "s/REPLACEME_HOSTNAME/`hostname`/" -e "s/REPLACEME_ASN/{routerasn}/" etc/frr/frr.conf-template > etc/frr/frr.conf',
    "cd vpp-frr/frr/ && sudo docker compose up -d",
    "sh vpp-frr/frr/keygen.sh"]
runcmd([routername], commands)

RUN COMMANDS
Execute: mkdir -p ~/.ssh on frr-cern
Execute: chmod 700 ~/.ssh on frr-cern
Execute local cmd: scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config  frrsshkey ubuntu@[2001:400:a100:3090:f816:3eff:fe7c:cf6]:~/.ssh/frrsshkey
(b'', b"Warning: Permanently added 'bastion.fabric-testbed.net' (ED25519) to the list of known hosts.\r\nWarning: Permanently added '2001:400:a100:3090:f816:3eff:fe7c:cf6' (ED25519) to the list of known hosts.\r\n")
Execute local cmd: scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config  frrsshkey.pub ubuntu@[2001:400:a100:3090:f816:3eff:fe7c:cf6]:~/.ssh/frrsshkey.pub
(b'', b"Warning: Permanently added 'bastion.fabric-testbed.net' (ED25519) to the list of known hosts.\r\nWarning: Permanently added '2001:400:a100:3090:f816:3eff:fe7c:cf6' (ED25519) to the list of known hosts.\r\n")
RUN COMMANDS
Execute: chmod 600 ~/.ssh/frrsshkey on frr-cern
RUN COMMANDS
Execute: sudo nft fl

In [24]:
slice = fablib.get_slice(name=slice_name)
# This is default routes, once VPP, FRR is running;
router = slice.get_node(name=routername)
# 1. Disable accept_ra for the first interface. That add a default route and we want to avoid that
router_iface = router.get_interface(network_name=f'pub-0')
iface = router_iface.get_physical_os_interface_name()
# 4. Set default routes for all our known ranges (CERN,Fabric,Caltech,SDSC,Fermilab,Nebraska, etc..)
for iprange, name in use_dataplane_routes.items():
    #command = f"sudo ip -6 route add {iprange} via {frr_config['gw']} dev {iface}.{frr_config['vlan']}"
    command = f"sudo ip -6 route add {iprange} via {frr_config['gw']} dev e2.99"
    print(command)
    runonecmd(router, routername, command)

sudo ip -6 route add 2602:fcfb::/36 via 2602:fcfb:001d:fff0::1 dev e2.99
Execute: sudo ip -6 route add 2602:fcfb::/36 via 2602:fcfb:001d:fff0::1 dev e2.99 on frr-cern
RTNETLINK answers: File exists
sudo ip -6 route add 2001:1458::/32 via 2602:fcfb:001d:fff0::1 dev e2.99
Execute: sudo ip -6 route add 2001:1458::/32 via 2602:fcfb:001d:fff0::1 dev e2.99 on frr-cern
RTNETLINK answers: File exists
sudo ip -6 route add 2620:6a::/48 via 2602:fcfb:001d:fff0::1 dev e2.99
Execute: sudo ip -6 route add 2620:6a::/48 via 2602:fcfb:001d:fff0::1 dev e2.99 on frr-cern
RTNETLINK answers: File exists
sudo ip -6 route add 2605:d9c0::/32 via 2602:fcfb:001d:fff0::1 dev e2.99
Execute: sudo ip -6 route add 2605:d9c0::/32 via 2602:fcfb:001d:fff0::1 dev e2.99 on frr-cern
RTNETLINK answers: File exists
sudo ip -6 route add 2600:900::/28 via 2602:fcfb:001d:fff0::1 dev e2.99
Execute: sudo ip -6 route add 2600:900::/28 via 2602:fcfb:001d:fff0::1 dev e2.99 on frr-cern
RTNETLINK answers: File exists
sudo ip -6 route

## Download SiteRM Repo to all nodes

In [ ]:
slice = fablib.get_slice(name=slice_name)
commands = ["sudo nft flush ruleset",
            "git clone https://github.com/sdn-sense/siterm-startup"]
node1 = slice.get_node(name=routername)
runcmd([routername], commands)
for host in list(hosts.keys()):
    runcmd([host], commands)

## Copy Certificates, creds from your bastion node

In [ ]:
#commands = ["cd ~/vpp-frr && git pull"]
#runcmd([routername], commands)
#print('-'*100)

# Install docker, clone tuning, and tune node;
commands = ["cd ~/vpp-frr && git pull"]
runcmd(list(hosts.keys()), commands)
print('-'*100)

# Generate and print commands to copy cert's to required locations
import os

if not os.path.isfile('macaroon-secret'):
    print('If you dont have macaroon secret, generate it using the following command')
    print('openssl rand -base64 -out macaroon-secret 64')
    print('This file will need to be equal between all nodes')
    print('-'*80)

scp_cmd = "scp -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config "

# frr node for SiteRM FE
# hosts - for SiteRM Agent;
#         for XRootD;
slice = fablib.get_slice(name=slice_name)
router = slice.get_node(name=routername)
user = router.get_username()
mgmtip = router.get_management_ip()
for dest, srcfile in {'~/siterm-startup/fe/conf/etc/httpd/certs/cert.pem': 'hostcert.pem',
                      '~/siterm-startup/fe/conf/etc/grid-security/hostcert.pem': 'hostcert.pem',
                      '~/siterm-startup/fe/conf/etc/httpd/certs/privkey.pem': 'hostkey.pem',
                      '~/siterm-startup/fe/conf/etc/grid-security/hostkey.pem': 'hostkey.pem'}.items():
    cmd = f"{scp_cmd} {srcfile} {user}@[{mgmtip}]:{dest}"
    print(localcmd(cmd))
# Print host commands
for hostname in list(hosts.keys()):
    node = slice.get_node(name=hostname)
    user = node.get_username()
    mgmtip = node.get_management_ip()
    for dest, srcfile in {'~/siterm-startup/agent/conf/etc/grid-security/hostcert.pem': 'hostcert.pem',
                          '~/siterm-startup/agent/conf/etc/grid-security/hostkey.pem': 'hostkey.pem',
                          '~/vpp-frr/xrootd/priv/xrootdcert.pem': 'hostcert.pem',
                          '~/vpp-frr/xrootd/priv/xrootdkey.pem': 'hostkey.pem',
                          '~/vpp-frr/xrootd/priv/macaroon-secret': 'macaroon-secret'}.items():
        cmd = f"{scp_cmd} {srcfile} {user}@[{mgmtip}]:{dest}"
        print(localcmd(cmd))


## Deploy SiteRM Frontend

In [ ]:
slice = fablib.get_slice(name=slice_name)
commands = ["cd siterm-startup && git pull",
            "cd vpp-frr && git pull",
            f"cp vpp-frr/siterm/{site}/fe/ansible-conf.yaml siterm-startup/fe/conf/etc/ansible-conf.yaml",
            f"cp vpp-frr/siterm/{site}/fe/siterm.yaml siterm-startup/fe/conf/etc/siterm.yaml",
            f"cp vpp-frr/siterm/{site}/fe/environment siterm-startup/fe/conf/environment",
            f'cp ~/.ssh/frrsshkey siterm-startup/fe/conf/opt/siterm/config/ssh-keys/frrsshkey',
            "cd siterm-startup/fe/docker/ && ./run.sh -i dev -n host"]
node1 = slice.get_node(name=routername)
runcmd([routername], commands)

## Restart SiteRM Frontend with new image (if needed)

In [ ]:
slice = fablib.get_slice(name=slice_name)
commands = ["cd siterm-startup && git pull",
            "cd siterm-startup/fe/docker/ && ./restart-new-image.sh -i dev -n host"]
node1 = slice.get_node(name=routername)
runcmd([routername], commands)

## Deploy SiteRM Agent(s)

In [ ]:
slice = fablib.get_slice(name=slice_name)
for host in list(hosts.keys()):
    commands = ["cd siterm-startup && git pull",
                "cd vpp-frr/ && git pull",
                f"cp vpp-frr/siterm/{site}/{host}/siterm.yaml siterm-startup/agent/conf/etc/siterm.yaml",
                 "cd siterm-startup/agent/docker/ && sudo ./run.sh -i dev"]
    runcmd([host], commands)

## Restart all SiteRM Agents with new image (if needed)

In [ ]:
slice = fablib.get_slice(name=slice_name)
for host in list(hosts.keys()):
    commands = ["cd siterm-startup && git pull",
                 "cd siterm-startup/agent/docker/ && sudo ./restart-new-image.sh -i dev"]
    runcmd([host], commands)

# Deploy XRootD on all Hosts

## Configure certs, users, dir and config file for xrootd;

In [ ]:
slice = fablib.get_slice(name=slice_name)
commands = ["cd ~/vpp-frr && git pull"]
runcmd(list(hosts.keys()), commands)
print('-'*100)


commands = ["sudo python3 ~/vpp-frr/xrootd/config/default/opt/usergroupchecker.py --notimeout",
            "sudo chown xrootd:xrootd ~/vpp-frr/xrootd/priv/*.pem",
            "sudo chmod 600 ~/vpp-frr/xrootd/priv/xrootdkey.pem",
            "sudo mkdir -p /storage/cms/store/",
            "sudo chown cmsuser:cmsuser /storage/cms/store/",
            "sed -i 's|XRD_HTTP_SECRET_KEY=.*|XRD_HTTP_SECRET_KEY=%s|' ~/vpp-frr/xrootd/.env" % xrd_http_secret,
            "sed -i 's|XRD_REDIR=.*|XRD_REDIR=%s|' ~/vpp-frr/xrootd/.env" % xrd_redir]

for host in list(hosts.keys()):
    runcmd([host], commands)

# Start XRootD Process on all nodes;

In [ ]:
commands = ["cd vpp-frr/xrootd/ && sudo docker compose up -d"]
for host in list(hosts.keys()):
    runcmd([host], commands)

# Install FDT

In [ ]:
slice = fablib.get_slice(name=slice_name)
for host in list(hosts.keys()):
    commands = ["sudo apt update", "sudo apt install -y default-jre",
                "sudo nft flush ruleset",
                "sudo ip -6 r add 2001:48d0:3001:111::/64 via 2602:fcfb:0012:fff2::1",
                "sudo ip -6 r add 2001:48d0:3001:112::/64 via 2602:fcfb:001d:fff3::1",
                "wget --no-check-certificate http://monalisa.cern.ch/FDT/lib/fdt.jar"]
    runcmd([host], commands)

# Run FDT in nohup

In [ ]:
slice = fablib.get_slice(name=slice_name)
for host in list(hosts.keys()):
    commands = ["sudo nft flush ruleset", "nohup java -jar fdt.jar &"]
    #commands = ["sudo ip -6 r add 2001:48d0:3001:111::/64 via 2602:fcfb:1d:fff2::1",
    #            "sudo ip -6 r add 2001:48d0:3001:112::/64 via 2602:fcfb:1d:fff3::1"]
    runcmd([host], commands)

## Extend slice to max (2 weeks)

Extend slice for another two weeks to keep it active

In [3]:
site="CERN"
# Extend slice (if already present)
from datetime import datetime
from datetime import timezone
from datetime import timedelta
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

#Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice_name = f'FRR-{site.lower()}'
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_slivers()
    slice.list_interfaces()
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")
 
# Check slice end date
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End (UTC)        : {slice.get_lease_end()}")
        
except Exception as e:
    print(f"Exception: {e}")


Retry: 1, Time: 48 sec


ID,c696ae75-d8e0-4f22-8ed1-6e8305ea411e
Name,FRR-cern
Lease Expiration (UTC),2025-04-10 23:05:46 +0000
Lease Start (UTC),2025-02-06 16:50:22 +0000
Project ID,a57c7715-d871-4369-82e6-408c9a57a6e7
State,StableOK
Email,jbalcas@es.net
UserId,91f5ecc3-16ff-4f09-95ac-dfeee0c3b1e3


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
b3c9e942-aabb-445a-8b31-e6418df751be,frr-cern,16,32,100,docker_ubuntu_22,qcow2,cern-w3.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe7c:cf6,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe7c:cf6,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b431107f-b974-424c-9238-59926da91dba,host1,16,32,1000,docker_ubuntu_22,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe1a:2102,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe1a:2102,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
743f4c2b-76c5-4097-ac0e-50c2b3e0c3ce,host2,16,32,1000,docker_ubuntu_22,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe18:71b0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe18:71b0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2d9ea70f-ea7f-48a5-98f6-f9dfd4d4c3b7,host3,16,32,1000,docker_ubuntu_22,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe0d:7260,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe0d:7260,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
08009764-9076-4a4d-88f0-6ac9828658c6,host4,16,32,1000,docker_ubuntu_22,qcow2,cern-w2.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe99:d011,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe99:d011,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a5287059-e3d6-459c-a3cf-fecf69b686ac,meas-node,4,16,100,docker_ubuntu_20,qcow2,cern-w6.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe7e:8699,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe7e:8699,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
59a0ceb0-2064-4256-b203-b6be0cafffa8,frr-net0,L2,L2Bridge,CERN,2602:fcfb:1d:fff1::/64,2602:fcfb:1d:fff1::1,Active,
55781c32-4107-41dd-87e4-c7e2899cbd5c,frr-net1,L2,L2Bridge,CERN,2602:fcfb:1d:fff2::/64,2602:fcfb:1d:fff2::1,Active,
8739c065-ad63-47a5-a33b-822f50baa48f,frr-net2,L2,L2Bridge,CERN,2602:fcfb:1d:fff3::/64,2602:fcfb:1d:fff3::1,Active,
c95f4ec9-bf7d-403b-bb0c-e981aefcb8e2,l3_meas_net_CERN,L3,FABNetv4,CERN,10.143.8.0/24,10.143.8.1,Active,
4f0b9837-c2e2-4603-a417-a64505ee9cf0,nic_local1_1,L2,L2Bridge,CERN,None,None,Active,
57ef41c0-a7fe-4b8e-8fbe-33c35bafda52,nic_local1_2,L2,L2Bridge,CERN,None,None,Active,
ee7cf8f2-cd50-4ca0-a1ef-70c10bd85dd1,pub-0,L3,FABNetv6Ext,CERN,2602:fcfb:1d:fff0::/64,2602:fcfb:1d:fff0::1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
frr-cern-nic_local1-p1,p1,frr-cern,nic_local1_1,100,config,,10:70:FD:E5:B2:68,enp8s0np0,enp8s0np0,fe80::1270:fdff:fee5:b268,1,HundredGigE0/0/0/8
frr-cern-nic_local1-p2,p2,frr-cern,nic_local1_2,100,config,,10:70:FD:E5:B2:69,enp9s0np0,enp9s0np0,fe80::1270:fdff:fee5:b269,1,HundredGigE0/0/0/10
frr-cern-nic_local-p1,p1,frr-cern,None,100,config,,10:70:FD:E5:A2:E8,enp10s0np0,enp10s0np0,fe80::1270:fdff:fee5:a2e8,6,None
frr-cern-nic_local-p1-vlan102,frr-cern-nic_local-p1-vlan102,frr-cern,frr-net2,100,manual,102,10:70:FD:E5:A2:E8,enp10s0np0,enp10s0np0.102,None,6,HundredGigE0/0/0/12
frr-cern-nic_local-p1-vlan100,frr-cern-nic_local-p1-vlan100,frr-cern,frr-net0,100,manual,100,10:70:FD:E5:A2:E8,enp10s0np0,enp10s0np0.100,None,6,HundredGigE0/0/0/12
frr-cern-nic_local-p1-vlan101,frr-cern-nic_local-p1-vlan101,frr-cern,frr-net1,100,manual,101,10:70:FD:E5:A2:E8,enp10s0np0,enp10s0np0.101,None,6,HundredGigE0/0/0/12
frr-cern-nic_local-p2,p2,frr-cern,None,100,config,,10:70:FD:E5:A2:E9,enp11s0np0,enp11s0np0,fe80::1270:fdff:fee5:a2e9,6,None
frr-cern-nic_local-p2-vlan99,frr-cern-nic_local-p2-vlan99,frr-cern,pub-0,100,manual,99,10:70:FD:E5:A2:E9,enp11s0np0,enp11s0np0.99,None,6,HundredGigE0/0/0/14
frr-cern-meas_nic_frr-cern_CERN-p1,p1,frr-cern,l3_meas_net_CERN,100,auto,,02:D3:AC:B1:7A:8C,enp7s0,enp7s0,10.143.8.7,4,HundredGigE0/0/0/9
host1-host1_n3-p1,p1,host1,frr-net2,100,manual,,12:98:C7:A2:F7:CE,enp9s0,enp9s0,2602:fcfb:1d:fff3::2,6,HundredGigE0/0/0/5



Time to print interfaces 116 seconds
Lease End (UTC)        : 2025-04-10 23:05:46 +0000


## Print slice SSH Commands

In [ ]:
site="CERN"
slice_name = f'FRR-{site.lower()}'
slice=fablib.get_slice(slice_name)
slice.list_nodes();
slice.list_interfaces();